# Planning strategies

In [1]:
from utilities.std_imports import *
import pulp
import supply_chain.libs.scanalytics as sca
import supply_chain.libs.wagner_whitin as ww

### Generate data

In [3]:
dem_fcst = [1040,240,480,400,1600,4400,1440,1120,480,400,800,2000]
setup_cost = 1822.5
holding_cost = 0.3375
init_inventory = 0

In [56]:
def show_plan(status='', inventory=[], planned=[], cost=0):
    plan = pd.DataFrame(columns=['t1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12'])
    plan.loc[0] = dem_fcst
    plan.loc[1] = inventory
    plan.loc[2] = planned
    plan.index = ['fcst', 'stock', 'planned']
    display(plan)
    print('')
    print('Status:\t', status) 
    print('Cost: $\t', cost)

### Lot for Lot (LFL)
Chase. No inventory, just planned orders

In [57]:
inventory, planned, cost = sca.MPS_chase(dem_fcst,setup_cost,holding_cost,init_inventory)
show_plan(status, inventory, planned, cost)

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12
fcst,1040,240,480,400,1600,4400,1440,1120,480,400,800,2000
stock,0,0,0,0,0,0,0,0,0,0,0,0
planned,1040,240,480,400,1600,4400,1440,1120,480,400,800,2000



Status:	 Optimal
Cost: $	 21870.0


### One time run
Calculate order as sum of demand on fcst horizon for one time order

In [65]:
inventory,planned,cost = sca.MPS_onetime(demand_forecast,setup_cost,holding_cost,init_inventory)
show_plan(status, inventory, planned, cost)

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12
fcst,1040,240,480,400,1600,4400,1440,1120,480,400,800,2000
stock,13360,13120,12640,12240,10640,6240,4800,3680,3200,2800,2000,0
planned,14400,0,0,0,0,0,0,0,0,0,0,0



Status:	 Optimal
Cost: $	 30415.5


### Fixed Order Quantity (FOQ)
Fixed lot. When inventory becomes zero it triggers the fixed order 

In [66]:
Q = 3600
inventory,planned,cost = sca.MPS_FOQ(Q, dem_fcst, setup_cost, holding_cost, init_inventory)
show_plan(status, inventory, planned, cost)

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12
fcst,1040,240,480,400,1600,4400,1440,1120,480,400,800,2000
stock,2560,2320,1840,1440,3440,2640,1200,80,3200,2800,2000,0
planned,3600,0,0,0,3600,3600,0,0,3600,0,0,0



Status:	 Optimal
Cost: $	 15228.0


### POQ (Periodic Order Quantity)
Fixed Period (so fixed frequency) of orders

In [67]:
t = 3
inventory, planned, cost = sca.MPS_POQ(t, dem_fcst, setup_cost, holding_cost, init_inventory)
show_plan(status, inventory, planned, cost)

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12
fcst,1040,240,480,400,1600,4400,1440,1120,480,400,800,2000
stock,720,480,0,6000,4400,0,1600,480,0,2800,2000,0
planned,1760,0,0,6400,0,0,3040,0,0,3200,0,0



Status:	 Optimal
Cost: $	 13527.0


### Silver meal

In [58]:
inventory, planned, cost = sca.MPS_silvermeal(dem_fcst, setup_cost, holding_cost, init_inventory)
show_plan(status, inventory, planned, cost)

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12
fcst,1040,240,480,400,1600,4400,1440,1120,480,400,800,2000
stock,1120,880,400,0,7840,3440,2000,880,400,0,2000,0
planned,2160,0,0,0,9440,0,0,0,0,0,2800,0



Status:	 Optimal
Cost: $	 11866.5


### Mixed Integer Linear Programming

In [43]:
status, inventory, planned, cost = sca.MPS_MILP(dem_fcst, setup_cost, holding_cost, init_inventory)
show_plan(status, inventory, planned, cost)

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12
stock,1120.0,880.0,400.0,0.0,0.0,3440.0,2000.0,880.0,400.0,0.0,2000.0,0.0
planned,2160.0,0.0,0.0,0.0,1600.0,7840.0,0.0,0.0,0.0,0.0,2800.0,0.0


Status:	 Optimal
Cost: $	 11043.0


### Wagner-Whitin

In [4]:
ww.wagner_whitin(setup_cost, holding_cost, dem_fcst)

n:  12
==========================constructing cost matrix==========================
cost matrix:  [[1822.5, 1903.5, 2227.5, 2632.5, 4792.5, 12217.5, 15133.5, 17779.5, 19075.5, 20290.5, 22990.5, 30415.5], [0, 1822.5, 1984.5, 2254.5, 3874.5, 9814.5, 12244.5, 14512.5, 15646.5, 16726.5, 19156.5, 25906.5], [0, 0, 1822.5, 1957.5, 3037.5, 7492.500000000001, 9436.500000000002, 11326.500000000002, 12298.500000000002, 13243.500000000002, 15403.500000000002, 21478.5], [0, 0, 0, 1822.5, 2362.5, 5332.5, 6790.5, 8302.5, 9112.5, 9922.5, 11812.5, 17212.5], [0, 0, 0, 0, 1822.5, 3307.5, 4279.5, 5413.5, 6061.5, 6736.5, 8356.5, 13081.5], [0, 0, 0, 0, 0, 1822.5, 2308.5, 3064.5, 3550.5, 4090.5, 5440.5, 9490.5], [0, 0, 0, 0, 0, 0, 1822.5, 2200.5, 2524.5, 2929.5, 4009.5, 7384.5], [0, 0, 0, 0, 0, 0, 0, 1822.5, 1984.5, 2254.5, 3064.5, 5764.5], [0, 0, 0, 0, 0, 0, 0, 0, 1822.5, 1957.5, 2497.5, 4522.5], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1822.5, 2092.5, 3442.5], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1822.5, 2497.5], [0, 0, 0, 0

(11043.0, [2160, 0, 0, 0, 1600, 7840, 0, 0, 0, 0, 2800, 0])

### Other methods
- Least Unit Cost
- Calendar
- Part Period Balancing

### Credits & Links

https://github.com/ragamarkely/scanalytics  
https://scholarworks.uark.edu/cgi/viewcontent.cgi?article=2029&context=etd  
https://6river.com/guide-to-warehouse-inventory-replenishment/#Models  
https://github.com/ichigooo/wagner-whitin/blob/master/wagner_whitin.py
https://en.wikipedia.org/wiki/Dynamic_lot-size_model